<a href="https://colab.research.google.com/github/gitHubAndyLee2020/Monte_Carlo_Chess_Agent/blob/main/monte_carlo_chess_agent_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Chess Agent using Monte Carlo Search Tree Algorithm

In [3]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.3 MB/s eta 0:00:00


In [4]:
import chess
import chess.pgn
import chess.engine
import random
import time
from math import log, sqrt, e, inf

In [5]:
class node():
  def __init__(self):
    self.state = chess.Board()
    self.action = ''
    self.children = set()
    self.parent = None
    self.N = 0
    self.n = 0
    self.v = 0

In [21]:
# Determines how valuate a node is to be explored
def ucb1(curr_node):
    ans = curr_node.v+2*(sqrt(log(curr_node.N+e+(10**-6))/(curr_node.n+(10**-10))))
    return ans

In [19]:
# Handles leaf node case, make random moves until the game is over
def rollout(curr_node):
  # Return reward value if the game is over
  if (curr_node.state.is_game_over()):
    board = curr_node.state
    if (board.result() == '1-0'):
      print("h1")
      return (1, curr_node)
    elif (board.result() == '0-1'):
      print("h2")
      return (-1, curr_node)
    else:
      return (0.5, curr_node)

  # If the game is not over, get list of all possible moves
  all_moves = [curr_node.state.san(i) for i in list(curr_node.state.legal_moves)]

  # Get child node for each possible move
  for i in all_moves:
    tmp_state = chess.Board(curr_node.state.fen())
    tmp_state.push_san(i)
    child = node()
    child.state = tmp_state
    child.parent = curr_node
    curr_node.children.add(child)

  # Get a random move
  rnd_state = random.choice(list(curr_node.children))

  return rollout(rnd_state)

In [12]:
# Select the most promising child node
def expand(curr_node, white):
  if (len(curr_node.children) == 0):
    return curr_node
  max_ucb = -inf
  if (white):
    idx = -1
    max_ucb = -inf
    sel_child = None
    for i in curr_node.children:
      tmp = ucb1(i)
      if (tmp > max_ucb):
        idx = i
        max_ucb = tmp
        sel_child = i

    return expand(sel_child, 0)

  else:
    idx = -1
    min_ucb = inf
    sel_child = None
    for i in curr_node.children:
      tmp = ucb1(i)
      if (tmp < min_ucb):
        idx = i
        min_ucb = tmp
        sel_child = i

    return expand(sel_child, 1)

In [11]:
def rollback(curr_node, reward):
  curr_node.n += 1
  curr_node.v += reward
  while (curr_node.parent != None):
    curr_node.N += 1
    curr_node = curr_node.parent
  return curr_node

In [18]:
def mcts_pred(curr_node, over, white, iterations=10):
  if (over):
    return -1

  all_moves = [curr_node.state.san(i) for i in list(curr_node.state.legal_moves)]
  map_state_move = dict()

  for i in all_moves:
    tmp_state = chess.Board(curr_node.state.fen())
    tmp_state.push_san(i)
    child = node()
    child.state = tmp_state
    child.parent = curr_node
    curr_node.children.add(child)
    map_state_move[child] = i

  # Expands the tree
  while (iterations > 0):
    if (white):
      idx = -1
      max_ucb = -inf
      sel_child = None
      # Select best child node
      for i in curr_node.children:
        tmp = ucb1(i)
        if (tmp > max_ucb):
          idx = i
          max_ucb = tmp
          sel_child = i
      # Go down the tree until leaf node and play until the end of the game
      ex_child = expand(sel_child, 0)
      reward, state = rollout(ex_child)
      curr_node = rollback(state, reward)
      iterations -= 1

    else:
      idx = -1
      min_ucb = inf
      sel_child = None
      for i in curr_node.children:
        tmp = ucb1(i)
        if (tmp < min_ucb):
          idx = i
          min_ucb = tmp
          sel_child = i

      ex_child = expand(sel_child, 1)
      reward, state = rollout(ex_child)
      curr_node = rollback(state, reward)
      iterations -= 1

  # Selects the most promising child for the next move
  if (white):
    mx = -inf
    idx = -1
    selected_move = ''
    for i in (curr_node.children):
      tmp = ucb1(i)
      if (tmp > mx):
        mx = tmp
        selected_move = map_state_move[i]
    return selected_move

  else:
    mn = inf
    idx = -1
    selected_move = ''
    for i in (curr_node.children):
      tmp = ucb1(i)
      if (tmp < mn):
        mn = tmp
        selected_move = map_state_move[i]
    return selected_move

In [15]:
!apt-get install -y stockfish

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  polyglot xboard | scid
The following NEW packages will be installed:
  stockfish
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 24.8 MB of archives.
After this operation, 47.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 stockfish amd64 14.1-1 [24.8 MB]
Fetched 24.8 MB in 1s (40.7 MB/s)
Selecting previously unselected package stockfish.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../stockfish_14.1-1_amd64.deb ...
Unpacking stockfish (14.1-1) ...
Setting up stockfish (14.1-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [17]:
board = chess.Board()

In [16]:
engine = chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish")

In [22]:
white = 1
moves = 0
pgn = []
game = chess.pgn.Game()
sm = 0
cnt = 0
while ((not board.is_game_over())):
  all_moves = [board.san(i) for i in list(board.legal_moves)]
  start = time.time()
  root = node()
  root.state = board
  result = mcts_pred(root, board.is_game_over(), white)
  sm += (time.time() - start)
  board.push_san(result)
  print(result)
  pgn.append(result)
  white ^= 1
  cnt += 1
  moves += 1

print(board)
print(" ".join(pgn))
print()
print(board.result())
game.headers["Result"] = board.result()
print(game)
engine.quit()

h1
h2
c4
h1
h1
Nf6
a4
h1
h1
h2
Ng4
h1
h1
h3
h1
h2
d5
h2
h1
h1
h1
hxg4
b6
Ra2
h2
h2
Bb7
h1
h1
Rh5
h2
e6
h2
h1
h1
Rxh7
d4
h1
h1
Qc2
h2
Ba3
h2
h2
d3
g5
h1
h2
Nc3
h1
h2
h1
h1
a6
h1
h1
h1
Qd2
h1
Ra7
h2
Qxg5
h1
dxc3
h1
h1
Qe3
h2
h2
h2
Bxg2
h1
h1
Qe5
Be7
h2
f3
h2
Qd6
h1
Rh2
Qd8
Qxh8+
h1
Kd7
h1
h2
h2
h2
Kd1
h1
h1
Qxh8
h1
h1
Bxg2
Ke8
h2
e4
h1
h1
Bd8
c5
h2
Qh7
h2
h2
Bg5
h1
h2
a5
cxb6
h1
h2
f6
h2
Rh6
h1
e5
h2
h1
Bh3
h1
h1
Rb7
h1
h2
Bh4
h2
c5
h2
h2
h2
Be1
h2
h2
Na6
h2
Bd2
h1
Rc7
h1
h2
bxc7
h2
h2
Qh8
Rxf6
h1
Qf8
h1
h1
Be3
h1
h1
h1
Be7
Rg6
Nxc7
h2
h2
h1
Bg5
h1
h1
h1
Bxg5
h1
Ne2
Qf4
h2
h1
h2
h2
Ng1
h2
h2
h2
Qd2#
. . . . k . . .
. . n . . . . .
. . . . . . R .
p . p . p . b .
P . . . P . P .
. . p P . P . B
R P . q . . . .
. . . K . . N .
c4 Nf6 a4 Ng4 h3 d5 hxg4 b6 Ra2 Bb7 Rh5 e6 Rxh7 d4 Qc2 Ba3 d3 g5 Nc3 a6 Qd2 Ra7 Qxg5 dxc3 Qe3 Bxg2 Qe5 Be7 f3 Qd6 Rh2 Qd8 Qxh8+ Kd7 Kd1 Qxh8 Bxg2 Ke8 e4 Bd8 c5 Qh7 Bg5 a5 cxb6 f6 Rh6 e5 Bh3 Rb7 Bh4 c5 Be1 Na6 Bd2 Rc7 bxc7 Qh8 Rxf6 Qf8 Be3 Be7 Rg6 Nxc7 Bg5 Bxg5 Ne2 Qf

In [32]:
!pip install cairosvg